In [93]:
import tushare as ts
import pandas as pd
from datetime import datetime
import os.path
import time
import numpy as np

In [94]:
def update_data_or_not(fileName, year):
    import os
    yearEnd = datetime.now().year-1  #去年
    updateTime = 0
    if ((year==yearEnd and datetime.now().month >= 5) or year < yearEnd):
        updateTime = 30
    else:
        updateTime = 1
    updateTime = updateTime*24*60*60
    finfo = os.stat(fileName)
    return(( time.time() - finfo.st_mtime) > updateTime)
    

In [95]:
def get_data(dataType, year, q=4): #年报
    typeAll = {'profit', 'report', 'growth'}
    if (dataType not in typeAll):
        print('%s not in %s' %(type, typeAll,))
        return
    saveFileName = './data/%s%02d%s.csv' % (year,q, dataType)
    if os.path.isfile(saveFileName):
        df = pd.read_csv(saveFileName)#, encoding='gb18030')
        return df
    fun = eval('ts.get_%s_data' % dataType)
    df = fun(year, q)
    df.to_csv(saveFileName, index=False, encoding='utf-8')
    return df


In [96]:
dfs = {'profit':{}, 'report':{}, 'growth':{}}
types = {'profit', 'report', 'growth'}
yearEnd = datetime.now().year  #去年
for t in types:
    for y in range(yearEnd-5, yearEnd):
        dfs[t][y] = get_data(t, y).rename(columns={i: i+str(y-yearEnd+5) for i in get_data(t, y).columns if i not in {'code'}})
        #dfs[t][y]['code'] = dfs[t][y]['code'].astype(int)
        dfs[t][y]['code'] = dfs[t][y]['code'].apply(lambda x: '%06d' % int(x))
#         
#             

In [97]:
dfs_growth = pd.DataFrame(columns=['code'])
for i in range(yearEnd-5, yearEnd):
    dfs_growth = dfs_growth.merge(dfs['growth'][i], on=['code'], how='outer')

# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')

In [98]:
stockGDF = dfs_growth[(dfs_growth.nprg0>25)&(dfs_growth.nprg1>25)\
           &(dfs_growth.nprg2>25)&(dfs_growth.nprg3>25)]


In [99]:
price = ts.get_realtime_quotes(stockGDF['code'])
def real_price(x, y):
    return x if x > 0 else y

price['price'] = price['price'].astype(float)
price['pre_close'] = price['pre_close'].astype(float)
price['price_real'] = price.apply(lambda x: real_price(x['price'], x['pre_close']), axis=1)
#price['price_true'] = map(lambda x, y : , price['price'].astype(float), price['pre_close'].astype(float))
#price = price.drop(['price_true'], axis=1)

In [100]:
stockGDF = stockGDF.merge(price, on='code')

In [101]:
dfs_report = pd.DataFrame(columns=['code'])
for i in range(yearEnd-2, yearEnd):
    dfs_report = dfs_report.merge(dfs['report'][i], on=['code'], how='outer')
# dfs['profit'][2013].merge(dfs['profit'][2014], on=['code', 'name'], how='outer')
dfs_report = dfs_report.drop_duplicates()
dfs_report['eps'] = dfs_report.apply(lambda x: x['eps3'] if pd.isnull(x['eps4']) else x['eps4'], axis=1)

In [102]:

stockGDF = stockGDF.merge(dfs_report, on='code', how='left')

In [103]:
stockGDF[stockGDF.code == '002078']

,code,name0,mbrg0,nprg0,nav0,targ0,epsg0,seg0,name1,mbrg1,...,eps4,eps_yoy4,bvps4,roe4,epcf4,net_profits4,profits_yoy4,distrib4,report_date4,eps
32,002078,太阳纸业,4.6736,44.983,23.9247,4.7251,44.4444,28.9797,太阳纸业,-4.0129,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.42


In [104]:
stockGDF['nprg'] = stockGDF.apply(lambda x: x['nprg3'] if pd.isnull(x['nprg4']) else x['nprg4'], axis=1)

In [105]:
df_basics = ts.get_stock_basics().drop(['name'], axis=1)
stockGDF = stockGDF.merge(df_basics, left_on='code', right_index=True)

In [106]:

stockGDF['peg'] = round(stockGDF['pe']/stockGDF['nprg'],2)

In [107]:
stockGDF[['code', 'name','peg', 'pe', 'eps', 'nprg', 'price_real'] + ['nprg'+str(i) for i in range(5)]][stockGDF.peg < 0.4]

,code,name,peg,pe,eps,nprg,price_real,nprg0,nprg1,nprg2,nprg3,nprg4
6,600816,安信信托,0.21,15.69,1.71,76.1722,12.97,172.7590,266.0663,68.2561,76.1722,NaN
10,002019,亿帆医药,0.25,22.40,NaN,88.8034,21.69,126.2480,829.1151,52.1242,88.8034,NaN
22,300237,美晨生态,0.22,25.51,NaN,115.0827,15.46,61.6673,237.5191,106.3751,115.0827,NaN
24,002841,视源股份,0.32,34.65,2.04,109.2521,67.80,57.5867,37.6380,56.9934,109.2521,NaN
30,300253,卫宁健康,0.31,69.84,NaN,225.3914,7.74,49.4068,56.4341,31.6772,225.3914,NaN
32,002078,太阳纸业,0.31,16.55,0.42,53.3837,11.70,44.9830,72.0237,38.0570,53.3837,NaN
40,300296,利亚德,0.38,38.57,0.87,100.6575,20.00,37.8114,104.1679,102.7194,100.6575,NaN
43,300121,阳谷华泰,0.08,22.55,0.56,298.7958,13.30,34.3584,71.0576,45.7954,298.7958,NaN
45,300182,捷成股份,0.29,25.02,0.39,85.3874,9.99,33.3725,27.2774,102.7172,85.3874,NaN
46,002589,瑞康医药,0.11,20.39,1.01,177.6553,13.05,30.3360,26.0113,33.7655,177.6553,NaN
